<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7cd375fd65c6ec69136a3ae8736edcc1cc48464565f995f1e6f049532c65b31d
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-25 11:17:36
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.40 C
-------------------
Today PnL: -27.44 K (-0.2%)
Current PnL: -20.09 L (-13.51%)
CY Booked + Current PnL: -8.65 L (-5.82%)
-------------------
Total profit:  1.39 L
Total loss:  -21.48 L
-------------------
Total Booked + Current PnL: 18.33 L (15.06%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.2%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 85.96 L (61.62%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.33%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.61,59.0,H-MC,3.87,195990.0,31320.0,12014.0,0.81,19.02,6.13,26.31,89.0,2.61,1.41,28.52,X40N,NTT,AC
77,TTKPRESTIG,770.00,98.29,44.0,M-SC,2.90,85412.0,-15365.0,15460.0,0.71,-15.25,18.10,0.09,245.0,-0.99,0.61,12.21,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-45.32,55.0,H-MC,9.82,109228.0,8724.0,20382.0,-1.32,8.68,18.66,28.96,79.0,0.43,0.78,45.60,MH,ATH,METALS
84,VOLTAS,1530.00,-1.78,47.0,H-MC,3.06,206145.0,14403.0,23356.0,0.15,7.51,11.33,19.69,99.0,0.62,1.48,14.82,XY25,NTT,AC
57,RELIANCE,1533.00,-14.04,43.0,H-LC,4.30,214952.0,4586.0,24204.0,-0.37,2.18,11.26,13.68,37.0,0.19,1.54,19.06,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,0.96,43.0,H-SC,9.20,120280.0,-5017.0,63941.0,2.60,-4.00,53.16,47.03,141.0,-0.08,0.86,50.60,MH,ATH,POWER
66,SIS,528.00,2096.37,43.0,H-SC,4.45,88638.0,-22394.0,46003.0,1.73,-20.17,51.90,21.26,156.0,-0.49,0.64,19.76,OX40N,NTT,MISC
75,TITAGARH,1548.00,-2.98,54.0,H-SC,2.72,207343.0,-34834.0,145596.0,1.36,-14.38,70.22,45.74,158.0,-0.24,1.49,34.86,XY24,NTT,ENGINEERING
8,AWL,485.00,-65.61,47.0,H-MC,3.30,244480.0,-57271.0,218687.0,1.27,-18.98,89.45,53.50,116.0,-0.26,1.75,8.59,XY24,NTT,FMCG
32,HINDZINC,730.22,22.12,58.0,M-LC,2.41,199100.0,-5976.0,118544.0,1.06,-2.91,59.54,54.89,52.0,-0.05,1.43,20.48,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,48.0,L-SC,42.84,82560.0,-30989.0,71043.0,-3.61,-27.29,86.05,35.27,268.0,-0.44,0.59,102.07,XR,NTT,HOTELS
71,TANLA,1963.11,-16.63,64.0,H-SC,16.33,223074.0,-47687.0,377642.0,-2.66,-17.61,169.29,121.86,133.0,-0.13,1.60,74.51,AR,ATH,IT
73,TATAMOTORS,1065.00,-21.50,44.0,M-LC,2.32,247911.0,-28612.0,138681.0,-2.62,-10.35,55.94,39.81,54.0,-0.21,1.78,18.81,XY24,NTT,AUTO
17,CAMS,5211.76,-4.28,48.0,H-SC,2.29,108727.0,6723.0,37206.0,-2.58,6.59,34.22,43.06,122.0,0.18,0.78,24.83,X40N,ATH,MISC
55,RAJOOENG,143.10,-41.44,26.0,H-SC,13.47,88900.0,-13600.0,54202.0,-2.04,-13.27,60.97,39.61,114.0,-0.25,0.64,0.00,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.34,43.0,H-MC,2.11,135295.0,-493.0,43240.0,0.34,-0.36,31.96,31.48,107.0,-0.01,0.97,11.71,X40,ATH,INSURANCE
23,DABUR,735.0,-7.44,40.0,H-MC,6.86,250405.0,2337.0,106072.0,-0.29,0.94,42.36,43.70,102.0,0.02,1.80,15.52,XY24,BTT,FMCG
86,WIPRO,420.0,-13.79,40.0,M-LC,5.49,151249.0,304.0,109156.0,-0.26,0.20,72.17,72.51,53.0,0.00,1.08,6.24,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,50.0,H-SC,9.92,121365.0,-21114.0,40633.0,-0.13,-14.82,33.48,13.70,163.0,-0.52,0.87,43.24,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,98.29,44.0,M-SC,2.90,85412.0,-15365.0,15460.0,0.71,-15.25,18.10,0.09,245.0,-0.99,0.61,12.21,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,51.0,H-SC,1.28,221481.0,-48186.0,84517.0,0.33,-17.87,38.16,13.47,138.0,-0.57,1.59,12.02,XY24,NTT,PAINTS
18,CERA,9475.0,-22.43,36.0,H-SC,1.98,141346.0,-34557.0,76581.0,0.38,-19.65,54.18,23.89,149.0,-0.45,1.01,22.07,OX40N,NTT,CERAMICS
66,SIS,528.0,2096.37,43.0,H-SC,4.45,88638.0,-22394.0,46003.0,1.73,-20.17,51.90,21.26,156.0,-0.49,0.64,19.76,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.96,70.0,M-MC,10.57,236947.0,11985.0,155674.0,0.92,5.33,65.70,74.53,192.0,0.08,1.70,39.30,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.61,59.0,H-MC,3.87,195990.0,31320.0,12014.0,0.81,19.02,6.13,26.31,89.0,2.61,1.41,28.52,X40N,NTT,AC
84,VOLTAS,1530.00,-1.78,47.0,H-MC,3.06,206145.0,14403.0,23356.0,0.15,7.51,11.33,19.69,99.0,0.62,1.48,14.82,XY25,NTT,AC
17,CAMS,5211.76,-4.28,48.0,H-SC,2.29,108727.0,6723.0,37206.0,-2.58,6.59,34.22,43.06,122.0,0.18,0.78,24.83,X40N,ATH,MISC
23,DABUR,735.00,-7.44,40.0,H-MC,6.86,250405.0,2337.0,106072.0,-0.29,0.94,42.36,43.70,102.0,0.02,1.80,15.52,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-45.32,55.0,H-MC,9.82,109228.0,8724.0,20382.0,-1.32,8.68,18.66,28.96,79.0,0.43,0.78,45.60,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-43.19,37.0,M-SC,1.57,96000.0,4502.0,74246.0,-1.23,4.92,77.34,86.07,223.0,0.06,0.69,39.41,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,64.0,H-MC,7.89,185252.0,5444.0,106965.0,-0.45,3.03,57.74,62.52,88.0,0.05,1.33,38.15,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.01,26.0,H-SC,10.46,125727.0,2391.0,131372.0,-0.19,1.94,104.49,108.45,119.0,0.02,0.90,24.72,AR,ATH,MISC
86,WIPRO,420.00,-13.79,40.0,M-LC,5.49,151249.0,304.0,109156.0,-0.26,0.20,72.17,72.51,53.0,0.00,1.08,6.24,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-19.44,34.0,H-LC,7.96,290003.0,-25839.0,138505.0,0.43,-8.18,47.76,35.67,5.0,-0.19,2.08,4.54,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-15.68,36.0,H-LC,7.41,212925.0,-38843.0,92047.0,-1.53,-15.43,43.23,21.13,27.0,-0.42,1.53,14.53,X40,ATH,PAINTS
74,TCS,4476.75,-29.01,37.0,H-LC,12.08,282630.0,-63326.0,138178.0,-0.95,-18.30,48.89,21.64,1.0,-0.46,2.03,0.12,X40,ATH,IT
42,ITC,452.00,-39.61,37.0,H-LC,2.09,195649.0,-4489.0,25376.0,-0.22,-2.24,12.97,10.44,4.0,-0.18,1.40,3.20,X40,NTT,FMCG
44,JIOFIN,387.00,-2.69,39.0,H-LC,13.15,264407.0,-2958.0,68402.0,0.23,-1.11,25.87,24.48,48.0,-0.04,1.90,53.26,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.20,48.0,H-LC,1.68,235350.0,-10801.0,89998.0,0.44,-4.39,38.24,32.18,16.0,-0.12,1.69,30.07,X200,ATH,IT
42,ITC,452.0,-39.61,37.0,H-LC,2.09,195649.0,-4489.0,25376.0,-0.22,-2.24,12.97,10.44,4.0,-0.18,1.40,3.20,X40,NTT,FMCG
65,SIEMENS,4671.5,-0.47,50.0,H-LC,2.81,160040.0,-26055.0,73538.0,-0.01,-14.00,45.95,25.51,15.0,-0.35,1.15,17.94,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.04,43.0,H-LC,4.30,214952.0,4586.0,24204.0,-0.37,2.18,11.26,13.68,37.0,0.19,1.54,19.06,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-10.64,46.0,H-LC,4.68,275161.0,9735.0,47053.0,-0.23,3.67,17.10,21.40,11.0,0.21,1.97,11.74,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-29.01,37.0,H-LC,12.08,282630.0,-63326.0,138178.0,-0.95,-18.30,48.89,21.64,1.0,-0.46,2.03,0.12,X40,ATH,IT
42,ITC,452.00,-39.61,37.0,H-LC,2.09,195649.0,-4489.0,25376.0,-0.22,-2.24,12.97,10.44,4.0,-0.18,1.40,3.20,X40,NTT,FMCG
83,VBL,671.64,-19.44,34.0,H-LC,7.96,290003.0,-25839.0,138505.0,0.43,-8.18,47.76,35.67,5.0,-0.19,2.08,4.54,X40N,ATH,FMCG
80,UNITDSPR,1644.00,-11.66,57.0,H-LC,7.10,231426.0,-2718.0,51353.0,-0.40,-1.16,22.19,20.77,86.0,-0.05,1.66,5.68,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.51,41.0,H-LC,9.86,224447.0,-17477.0,80756.0,-0.88,-7.22,35.98,26.15,8.0,-0.22,1.61,6.06,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-0.47,50.0,H-LC,2.81,160040.0,-26055.0,73538.0,-0.01,-14.00,45.95,25.51,15.0,-0.35,1.15,17.94,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,37.0,H-LC,2.09,195649.0,-4489.0,25376.0,-0.22,-2.24,12.97,10.44,4.0,-0.18,1.40,3.20,X40,NTT,FMCG
19,CIPLA,1795.00,-21.49,43.0,H-LC,4.93,209720.0,5220.0,36198.0,-0.47,2.55,17.26,20.25,10.0,0.14,1.50,11.51,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-15.68,36.0,H-LC,7.41,212925.0,-38843.0,92047.0,-1.53,-15.43,43.23,21.13,27.0,-0.42,1.53,14.53,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.04,43.0,H-LC,4.30,214952.0,4586.0,24204.0,-0.37,2.18,11.26,13.68,37.0,0.19,1.54,19.06,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-29.01,37.0,H-LC,12.08,282630.0,-63326.0,138178.0,-0.95,-18.30,48.89,21.64,1.0,-0.46,2.03,0.12,X40,ATH,IT
40,INFY,2275.00,-18.16,49.0,H-LC,8.07,319010.0,6122.0,165566.0,0.21,1.96,51.90,54.87,3.0,0.04,2.29,8.74,X40,BTT,IT
42,ITC,452.00,-39.61,37.0,H-LC,2.09,195649.0,-4489.0,25376.0,-0.22,-2.24,12.97,10.44,4.0,-0.18,1.40,3.20,X40,NTT,FMCG
83,VBL,671.64,-19.44,34.0,H-LC,7.96,290003.0,-25839.0,138505.0,0.43,-8.18,47.76,35.67,5.0,-0.19,2.08,4.54,X40N,ATH,FMCG
1,ABB,7934.00,-39.77,49.0,H-LC,8.27,251808.0,-9811.0,129026.0,0.17,-3.75,51.24,45.57,7.0,-0.08,1.81,7.15,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6766.67,50.0,L-SC,5.39,75337.0,-18473.0,95090.0,-0.88,-19.69,126.22,81.67,269.0,-0.19,0.54,45.89,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,48.0,L-SC,42.84,82560.0,-30989.0,71043.0,-3.61,-27.29,86.05,35.27,268.0,-0.44,0.59,102.07,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.69,41.0,H-SC,11.45,86838.0,-14125.0,108652.0,-0.20,-13.99,125.12,93.63,148.0,-0.13,0.62,29.34,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.89,50.0,H-SC,12.69,94200.0,-3780.0,25387.0,-0.68,-3.86,26.95,22.05,152.0,-0.15,0.68,36.93,XR,ATH,FINANCE
50,NATIONALUM,244.55,-45.32,55.0,H-MC,9.82,109228.0,8724.0,20382.0,-1.32,8.68,18.66,28.96,79.0,0.43,0.78,45.60,MH,ATH,METALS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1201.44,69.0,M-SC,11.45,177068.0,-4075.0,49544.0,-1.18,-2.25,27.98,25.10,235.0,-0.08,1.27,41.90,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-16.63,64.0,H-SC,16.33,223074.0,-47687.0,377642.0,-2.66,-17.61,169.29,121.86,133.0,-0.13,1.60,74.51,AR,ATH,IT
7,ATULAUTO,844.00,3689.19,53.0,M-SC,6.60,118106.0,-29262.0,81080.0,-1.43,-19.86,68.65,35.16,236.0,-0.36,0.85,20.63,XY24,NTT,AUTO
60,SAIL,228.00,49.96,70.0,M-MC,10.57,236947.0,11985.0,155674.0,0.92,5.33,65.70,74.53,192.0,0.08,1.70,39.30,XY24,BTT,STEEL
81,VAIBHAVGBL,521.00,59.23,58.0,H-SC,5.94,137819.0,-44956.0,156025.0,-0.39,-24.60,113.21,60.77,125.0,-0.29,0.99,23.44,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.28
1,25,43.78
2,50,74.90


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.93
LC    32.46
MC    23.65
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.30
X40      14.77
X40N     11.40
XR       11.24
XY25     10.12
AR        8.36
OX40N     7.86
X200      1.69
MH        1.64
X5K       1.43
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.41
H-LC    25.99
H-MC    20.68
M-SC    13.03
M-LC     5.42
M-MC     2.67
L-SC     1.49
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.97,-5.36,40.13
IT,12.97,-16.32,79.75
FINANCE,10.06,-16.92,67.93
MISC,6.98,-18.10,83.76
BANKS,6.16,-16.13,77.22
PAINTS,5.64,-17.46,34.97
ELECTRICAL,5.49,-7.66,45.43
INSURANCE,3.76,-5.70,45.21
AC,3.59,3.02,13.22


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3155780.0,22
XR,1342688.0,14
AR,1168821.0,9
X40,791154.0,10
X40N,614253.0,9
OX40N,550758.0,10
XY25,429549.0,7
SR,249750.0,2
X5K,118544.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3692490.0,29
M-SC,1431595.0,17
H-MC,1224403.0,15
H-LC,1197203.0,15
M-LC,393594.0,4
M-MC,304874.0,2
L-SC,252735.0,3
L-MC,60379.0,1
L-LC,38345.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1232533.0      6
M-SC       XY24       819393.0      7
H-SC       AR         797705.0      5
           XR         779546.0      7
H-MC       XY24       564684.0      4
H-LC       X40        501923.0      5
H-SC       X40N       309953.0      4
M-MC       XY24       304874.0      2
H-SC       OX40N      259062.0      4
           SR         249750.0      2
H-LC       X40N       226056.0      3
H-MC       X40        215153.0      4
H-LC       AR         202564.0      2
H-MC       XY25       169186.0      2
M-SC       AR         168552.0      2
L-SC       XR         166133.0      2
M-LC       XY24       165894.0      2
M-SC       OX40N      135305.0      4
           XR         120509.0      2
M-LC       X5K        118544.0      1
M-SC       XY25       113758.0      1
M-LC       XR         109156.0      1
H-LC       XY25       108260.0      3
H-MC       XR         106965.0      1
H-LC       X200        89998.0      1
L-SC       OX40N       86602.0      1
H-MC       X40N        78244.0      2
M-SC       X40         74078.0      1
H-MC       OX40N       69789.0      1
H-LC       XY24        68402.0      1
H-SC       MH          63941.0      1
L-MC       XR          60379.0      1
L-LC       XY25        38345.0      1
H-MC       MH          20382.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
